# Azure Cognitive Search LangChain Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search LangChain Vector Store
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents==11.4.0b6` until the Langchain AzureSearch vector store integration gets updated with the latest python azure sdk updates`.

In [ ]:
! pip install azure-search-documents==11.4.0b6
! pip install openai
! pip install python-dotenv
! pip install langchain

## Import required libraries and environment variables

In [2]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch


## Configure OpenAI Settings

In [5]:
# Configure environment variables  
load_dotenv()  
openai.api_type: str = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model: str = "text-embedding-ada-002"

## Configure Vector Store Settings

In [6]:
vector_store_address: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
index_name: str = "langchain-vector-demo"

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [7]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, model=model, chunk_size=1, openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"), openai_api_type="azure" )
index_name: str = "langchain-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

## Insert text and embeddings into vector store

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['YjBiNjUzZDItNGMzZC00OGM1LTgyYWQtNGQyODZkZjk0NzY3',
 'ZTdhNDdhYzAtZTI0My00ZTVmLWI2ZjktM2ZmYjcyOThkOGQ1',
 'YzdmOGUyMmQtZWQ4Ny00YjRkLWI0NWItZmIxYmNiZjliMTRi',
 'ZDMwM2ZkMmUtOTdhMi00NjQ2LThjNTktYTIyZDFmMGZmMDVm',
 'Mjc4ZTk3YWYtMWU3Yy00OWYyLThiMjEtOGMzYWU3ZmU2NzNh',
 'Mzk5ODcxZDktZjI3Ny00YzA5LWE5ZTUtNGI4OTdiZWRlMTdl',
 'Zjk5MDk2OGQtYjE3NS00NDcwLWI5ZTUtM2RhZDY3MzNkZTlk',
 'YzIyMGJjZDUtODRiNi00YTk3LWFjYjYtZWNkNTgwNzU2NWI2',
 'ZDU5MTNlYmUtODQ0Yi00ZmJiLTkyNDEtNzE5NmRiYTcwN2Fm',
 'MTQ4MjdmOTctODNhNi00MWU2LWEyNDAtMGNhZDJiYTYzZTAw',
 'ZTg3NzJkMDQtYmZhMi00ZjgxLTkyNDgtODVkZTJiZDUwYjEx',
 'OWQwODk3OTktMjFlMi00NGE1LTg1ZGItOGQzZTM3ZTliOTcw',
 'MDM4ZTI4NTUtYjA1ZC00MWU1LWIzZmYtYTM4YmQ3ZTM0MGVm',
 'N2U5ZWQxMDEtOGYzZS00NjUzLWJiMzgtNDk0MWM2MWI3N2I3',
 'ZjY4Y2QwYTEtOTJjOC00NDg0LWI5ZWYtNWRlZjAzMjk4Yjcw',
 'ZDIzNjY1YjktMWE2Zi00OTZjLThjN2ItNjY0MTM5ZTEyOGNk',
 'NTVmMzk5NGMtNWQ4Yi00NmNkLWE1NWYtM2M5ZmFlZWYyMmVk',
 'OTdhMzEzNjAtNzlmMy00MmFjLTliNmQtNTg1MTdkNmU0Mzhm',
 'NmRkNDYxNDYtMWFhYi00YTdjLTgxYzgtYTEyZjBjZjI4

## Perform a vector similarity search

In [9]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Perform a hybrid search

In [10]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
